In [19]:
import cv2 
import numpy as np 
import os 
import cv2 
import numpy as np 
import math
import time
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import os
from heapq import nsmallest
from skimage.filters import threshold_local
from skimage import measure
import imutils
from sklearn.cluster import KMeans



In [42]:
def delete_bbox_overlap(indexA, indexB,list_bbox,list_bbox_Copy):
    '''
    indexA: index of bbox A
    indexB: index of bbox B
    if bbox A and bbox B overlap, detele the smallest
    '''
    bboxA = list_bbox[indexA]
    bboxB = list_bbox[indexB]
    polygon_A = Polygon(bboxA)
    polygon_B = Polygon(bboxB)
    intersec_area = polygon_A.intersection(polygon_B).area
    area_polygonA = polygon_A.area
    area_polygonB = polygon_B.area
    ratio_overlap = intersec_area / min(area_polygonA,area_polygonB)
    if ratio_overlap >= 0.3 and area_polygonA < area_polygonB:
        if bboxA in list_bbox_Copy:
            list_bbox_Copy.remove(bboxA)
    elif ratio_overlap >= 0.3 and area_polygonA > area_polygonB:
        if bboxB in list_bbox_Copy:
            list_bbox_Copy.remove(bboxB)

def remove_bbox_noise(ls_bbox):
    '''
    Remove the bboxs that is too small or have special shape.
    
    '''
    try:
        ls_height = []
        for bbox in ls_bbox:
            ls_height.append(bbox[3][1] - bbox[0][1])
        index_bbox = []
        mean_height = sum(ls_height)/len(ls_height)
        for index in range(len(ls_height)):
            if mean_height*1.5 >= ls_height[index] or ls_height[index] >= mean_height*0.65:
                index_bbox.append(index)
        output_bbox = []
        for i in index_bbox:
            output_bbox.append(ls_bbox[i])
        return output_bbox
    except:
        return ls_bbox

def bbox_in_boundary_image(box,image):
    '''
    Checking if bbox has coordinates outside image
    '''
    count_zero = 0
    count_right = 0
    count_bottom = 0
    count_outside = 0
    for point in box:
        if 0 in point:
            count_zero += 1
        if point[1] >= image.shape[0] -2:
            count_outside += 1
        if point[0] >= image.shape[1] -2:
            count_right += 1
        if point[0] <0 or point[1] <0:
            count_outside += 1
            
    if count_zero >=2 or count_bottom >= 1 or count_right >= 1 or count_outside >=1:
        return False
    else:
        return True

In [46]:
import numpy as np
import cv2


def get_contour_precedence(contour, cols):
    tolerance_factor = 10
    origin = cv2.boundingRect(contour)
    return ((origin[1] // tolerance_factor) * tolerance_factor) * cols + origin[0]


def square(img):
    """
    This function resize non square image to square one (height == width)
    :param img: input image as numpy array
    :return: numpy array
    """

    # image after making height equal to width
    squared_image = img

    # Get image height and width
    h = img.shape[0]
    w = img.shape[1]

    # In case height superior than width
    if h > w:
        diff = h-w
        if diff % 2 == 0:
            x1 = np.zeros(shape=(h, diff//2))
            x2 = x1
        else:
            x1 = np.zeros(shape=(h, diff//2))
            x2 = np.zeros(shape=(h, (diff//2)+1))

        squared_image = np.concatenate((x1, img, x2), axis=1)

    # In case height inferior than width
    if h < w:
        diff = w-h
        if diff % 2 == 0:
            x1 = np.zeros(shape=(diff//2, w))
            x2 = x1
        else:
            x1 = np.zeros(shape=(diff//2, w))
            x2 = np.zeros(shape=((diff//2)+1, w))

        squared_image = np.concatenate((x1, img, x2), axis=0)

    return squared_image


def sort(vector):
    sort = True
    while (sort == True):

        sort = False
        for i in range(len(vector) - 1):
            x_1 = vector[i][0]
            y_1 = vector[i][1]

            for j in range(i + 1, len(vector)):

                x_2 = vector[j][0]
                y_2 = vector[j][1]

                if (x_1 >= x_2 and y_2 >= y_1):
                    tmp = vector[i]
                    vector[i] = vector[j]
                    vector[j] = tmp
                    sort = True

                elif (x_1 < x_2 and y_2 > y_1):
                    tmp = vector[i]
                    vector[i] = vector[j]
                    vector[j] = tmp
                    sort = True
    return vector



def plate_segmentation(img_file_path):

    img = cv2.imread(img_file_path)
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    height = img.shape[0]
    width = img.shape[1]
    area = height * width

    scale1 = 0.004
    scale2 = 0.1
    area_condition1 = area * scale1
    area_condition2 = area * scale2
    # global thresholding
    ret1,th1 = cv2.threshold(imgray,127,255,cv2.THRESH_BINARY)

    # Otsu's thresholding
    ret2,th2 = cv2.threshold(imgray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(imgray,(5,5),0)
    ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    contours, hierarchy = cv2.findContours(th3, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # sort contours
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    cropped = []
    list_bbox = []
    for cnt in contours:
        (x,y,w,h) = cv2.boundingRect(cnt)
        if (w * h > area_condition1 and w * h < area_condition2 and w/h > 0.3 and h/w > 0.3):
            list_bbox.append([(x,y),(x + w, y), (x + w, y + h), (x, y + h)])
            
    print("Shape of bbox raw:",np.shape(list_bbox))
    list_bbox_character = list_bbox.copy()
    for indexA in range(len(list_bbox) -1 ) :
        for indexB in range(indexA + 1, len(list_bbox)):
            delete_bbox_overlap(indexA, indexB, list_bbox, list_bbox_character)
            
    print("Shape bbox character", np.shape(list_bbox_character))
    list_bbox_character = remove_bbox_noise(list_bbox_character)
    print("Shape output bbox character", np.shape(list_bbox_character))
    for bbox in list_bbox_character:
        if (bbox_in_boundary_image(bbox,img)):
            cv2.rectangle(img, bbox[0], bbox[2], (255, 0, 0), 2)
    cv2.imwrite('detection.png', img)
    return cropped

In [47]:
cropped = plate_segmentation('/home/duongnh/Documents/10_image.jpg')


Shape of bbox raw: (12, 4, 2)
Shape bbox character (9, 4, 2)
Shape output bbox character (9, 4, 2)


In [41]:
print(np.shape(cropped))

(0,)
